In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [90]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [91]:
#feed.routes.route_type.iat[0] = 2  # Create another route type
trip_stats = feed.compute_trip_stats()

In [92]:
#dates = ["20100101"]
week = feed.get_first_week()
dates = [week[4], week[6]] #+ ["20100101"]
f = feed.compute_feed_stats(trip_stats, dates=dates, split_route_types=True)
f.T

,0,1
route_type,3,3
num_stops,416,411
num_routes,22,14
num_trips,636,266
num_trip_starts,636,266
num_trip_ends,617,263
service_distance,14290.4,6390.85
service_duration,483.017,197.683
service_speed,29.5858,32.3287
peak_num_trips,39,17


In [118]:
freq = "6H"
sts = feed.compute_stop_time_series(dates, freq=freq, split_directions=True)
rts = feed.compute_route_time_series(trip_stats, dates, freq=freq)
fts = feed.compute_feed_time_series(trip_stats, dates, freq=freq, split_route_types=True)

In [119]:
fts

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
route_type,3,3,3,3,3,3
datetime,,,,,,
2014-05-30 00:00:00,14.0,17.0,17.0,582.579000,12.450000,46.793494
2014-05-30 06:00:00,214.0,240.0,242.0,4954.224929,173.800000,28.505322
2014-05-30 12:00:00,248.0,253.0,281.0,5505.336088,193.216667,28.493070
2014-05-30 18:00:00,155.0,126.0,160.0,3248.283810,103.550000,31.369230
2014-05-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-31 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
(
    rts
    .pipe(gt.unstack_time_series)
    .merge(feed.routes.filter(["route_id", "route_type"]), how="left")
    .groupby(["datetime", "indicator", "route_type"])
    .agg({"value": lambda x: x.sum(skipna=True, min_count=1)})  # Preserve NaNs
    .reset_index("datetime")
#     .fillna(-1)
#     .pivot_table(
#         index=["datetime"], columns=["indicator", "route_type"]
#     )
#     .value
#     .replace({-1: np.nan})
)

,,datetime,value
indicator,route_type,,
num_trip_ends,3,2014-05-30 00:00:00,14.000000
num_trip_starts,3,2014-05-30 00:00:00,17.000000
num_trips,3,2014-05-30 00:00:00,17.000000
service_distance,3,2014-05-30 00:00:00,582.579000
service_duration,3,2014-05-30 00:00:00,12.450000
service_speed,3,2014-05-30 00:00:00,287.578550
num_trip_ends,3,2014-05-30 06:00:00,214.000000
num_trip_starts,3,2014-05-30 06:00:00,240.000000
num_trips,3,2014-05-30 06:00:00,242.000000


In [76]:
# Slicing 
display(fts.loc[:, ("num_trips", slice(None))])
display(fts.xs("num_trips", axis="columns"))

indicator           num_trips       
route_type                  2      3
datetime                            
2014-05-30 00:00:00      23.0  233.0
2014-05-30 12:00:00      40.0  368.0
2014-05-31 00:00:00       NaN    NaN
2014-05-31 12:00:00       NaN    NaN
2014-06-01 00:00:00       9.0    NaN
2014-06-01 12:00:00      25.0    NaN

route_type,2,3
datetime,,
2014-05-30 00:00:00,23.0,233.0
2014-05-30 12:00:00,40.0,368.0
2014-05-31 00:00:00,NaN,NaN
2014-05-31 12:00:00,NaN,NaN
2014-06-01 00:00:00,9.0,NaN
2014-06-01 12:00:00,25.0,NaN


In [77]:
f = fts = feed.compute_feed_time_series(trip_stats, dates[:1], freq="12H", split_route_types=False)
display(f)
g = gt.unstack_time_series(f)

# columns = [c for c in g.columns if c not in ["datetime", "value"]]
# h = g.pivot_table(index="datetime", columns=columns).value.sort_index(
#     axis="columns"
# )

# hours = (h.index[1] - h.index[0]).components.hours
# if hours != 0:
#     freqy = f"{hours}H"
# else:
#     freqy = "D"

# print(freqy)

h = gt.restack_time_series(g)
h

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2014-05-30 00:00:00,228,257,256,5536.803929,186.250000,29.727806
2014-05-30 12:00:00,403,379,408,8753.619898,296.766667,29.496641


indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2014-05-30 00:00:00,228.0,257.0,256.0,5536.803929,186.250000,29.727806
2014-05-30 12:00:00,403.0,379.0,408.0,8753.619898,296.766667,29.496641


In [68]:
f.index

DatetimeIndex(['2014-05-26 00:00:00', '2014-05-26 12:00:00'], dtype='datetime64[ns]', name='datetime', freq='12H')

In [66]:
delta.components.hours

NameError: name 'delta' is not defined

In [ ]:
feed1 = feed.copy()
cal = feed1.calendar.copy()
cal["monday"] = 0
feed1.calendar = cal
feed1.compute_feed_time_series(trip_stats, dates, freq="12H")

In [ ]:
s1 = pd.Series({"hello": 12, "goodbye": 13})
s2 = s1.copy()
pd.DataFrame([s1, s2])

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()